In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split

# Converting categorical to number
  # replacing values
dataset['smoker'].replace(['no', 'yes'],[0, 1], inplace=True)
dataset['sex'].replace(['male', 'female'],[0, 1], inplace=True)
dataset['region'].replace(['northwest', 'northeast', 'southeast', 'southwest'],[0, 1, 2, 3], inplace=True)

x_data = dataset.drop(columns=['expenses'])
y_data = dataset['expenses']
# split into input (X) and output (Y), train and test variables
train_dataset, test_dataset, train_labels, test_labels = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(train_dataset[:5])
print("Shapes: ", train_dataset.shape, test_dataset.shape, train_labels.shape, test_labels.shape)

In [ ]:
model = keras.Sequential()

num_features = len(train_dataset.keys())
#Creating the model
model.add(keras.layers.Dense(128, activation='relu', input_shape=(num_features,)))  # First hidden layer
layers.Dropout(.2), #20% dropout
model.add(keras.layers.Dense(64, activation='relu'))  # Second hidden layer
model.add(keras.layers.Dense(32, activation='relu'))  # Third hidden layer
model.add(keras.layers.Dense(1))  # Output layer with 1 unit for predicted price


In [ ]:
#COMPILING
model.compile(
    optimizer=tf.optimizers.Nadam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse']
)
model.summary()

In [ ]:
class CCallback(keras.callbacks.Callback):
  """
  Custom callback to track and print training progress with epochs, loss, MAE, and MSE.
  """
  def on_epoch_end(self, epoch, logs=None):
    if logs is None:
      return
    print(f"Epoch: {epoch+1}")
    print(f"Loss: {logs['loss']:.5f}")
    print(f"MAE: {logs['mae']:.5f}")
    print(f"MSE: {logs['mse']:.5f}")
    print("-" * 20)

#TRAINING

r = model.fit(train_dataset, train_labels, epochs=500,
              verbose=0,callbacks=[CCallback()])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
